In [1]:
def facility_path(instance):
    return 'documents/{}'.format(instance)

In [2]:
facility_path(12)

'documents/12'

In [ ]:
import docx
import pandas as pd
from os import path, listdir

In [ ]:
articles = pd.read_csv(r'jurnal_papers.csv')
articles.head()

,Article_ID,folder,ar_en,Country,Title,Author,Author_certificate,Author_specification,work,co_Author,...,Rev3_score,Score,Volume,Issue,Result,Date_of_editor_decision,artc_txt,clean_txt,clean2_txt,keywords
0,1,14-1,ar,Syria,تأثير طريقة الزراعة وعمق الحراثة في نمو الشوند...,فادي عباس,الدكتور,انتاج محاصيل (شوندر سكري),الهيئة العامة للبحوث العلمية الزراعية، مركز بح...,محمود السيدو,...,3.785714286,3.261904762,1,1,مقبول,NaN,NaN,NaN,NaN,NaN
1,3,14-3,ar,Syria,التأثير الطارد لمركب الكلس المكروني (Surround)...,مازن بوفاعور,المهندس,وقاية,الهيئة العامة للبحوث العلمية الزراعية، مركز بح...,"فيلي خاريزانوفا, ناريمان الزغبي",...,2,2.904761905,1,1,مقبول,NaN,NaN,NaN,NaN,NaN
2,4,14-4,ar,Syria,تأثير طريقة الري بالتنقيط (مغطى ومكشوف) في الا...,عبد الكريم جردي,المهندس,ري محاصيل حقلية,الهيئة العامة للبحوث العلمية الزراعية، مركز بح...,"بسام عودة, هيثم العبدالله, طلال العبدو, بشرى خزام",...,3,3.452380952,2,2,مقبول,11/27/2014,NaN,NaN,NaN,NaN
3,5,14-5,ar,Syria,تغيرات مجتمع حشرة بسيلا الأجاص Cacopsylla pyri...,بسام عودة,الدكتور,وقاية نبات (حشرات),الهيئة العامة للبحوث العلمية الزراعية، مركز بح...,"وجيه قسيس, رندة أبو طارة",...,NaN,2.892857143,2,1,مقبول,NaN,NaN,NaN,NaN,NaN
4,6,14-6,ar,Syria,تأثير سبعة أصول من الحمضيات في مواصفات النمو و...,علاء سهيل ابراهيم,المهندس,علوم البستنة,الهيئة العامة للبحوث العلمية الزراعية، مركز بح...,"إشراق سليمان علي, رائد صبيح, رفيق علي عبود, فا...",...,2.642857143,2.880952381,1,1,مقبول,NaN,NaN,NaN,NaN,NaN


In [ ]:
def read_docx(folder_path, folder_name):
    f_Text = []
    fText = ''
    try:
        for name in listdir(folder_path):
            if ((folder_name)== name[:-5]):
                filePath = path.join(folder_path, name)
                print('filePath is:{}'.format(filePath))
                doc = docx.Document(filePath)
                for line in doc.paragraphs:
                    f_Text.append(line.text)
            else:
                continue
    except:
        print("reading {} file failed, no such directory".format(folder_path))

    fText = '\n'.join(f_Text)
    return fText


In [ ]:
for i, a_id in enumerate(articles['folder']):
    link = r'D:\MWT\master_thesis\GCSAR\accepted\{}'.format(a_id)
    filetext = read_docx(link,a_id)
    articles.artc_txt[i] = filetext

filePath is:D:\MWT\master_thesis\GCSAR\accepted\14-1\14-1.docx


C:\Users\User\AppData\Local\Temp/ipykernel_11064/3217126970.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  articles.artc_txt[i] = filetext
C:\Users\User\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


filePath is:D:\MWT\master_thesis\GCSAR\accepted\14-3\14-3.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\14-4\14-4.docx
reading D:\MWT\master_thesis\GCSAR\accepted\14-5 file failed, no such directory
filePath is:D:\MWT\master_thesis\GCSAR\accepted\14-6\14-6.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\14-8\14-8.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\14-9\14-9.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\14-10\14-10.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\14-11\14-11.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\14-12\14-12.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\14-13\14-13.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\14-14\14-14.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\14-15\14-15.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\14-16\14-16.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\14-17\14-17.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\14-18\14-18.docx
filePath is:D:\MWT\m

filePath is:D:\MWT\master_thesis\GCSAR\accepted\17-179\17-179.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\17-181\17-181.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\17-184\17-184.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\17-185\17-185.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\17-187\17-187.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\17-190\17-190.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\17-191\17-191.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\17-192\17-192.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\17-194\17-194.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\17-195\17-195.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\17-197\17-197.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\17-198\17-198.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\17-201\17-201.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\17-202\17-202.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\17-206\17-206.

filePath is:D:\MWT\master_thesis\GCSAR\accepted\18-366\18-366.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\18-367\18-367.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\18-368\18-368.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\18-371\18-371.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\18-373\18-373.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\18-374\18-374.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\18-375\18-375.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\18-376\18-376.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\18-377\18-377.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\18-378\18-378.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\18-380\18-380.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\18-381\18-381.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\18-382\18-382.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\18-383\18-383.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\18-384\18-384.

filePath is:D:\MWT\master_thesis\GCSAR\accepted\19-538\19-538.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\19-542\19-542.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\19-547\19-547.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\19-548\19-548.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\19-549\19-549.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\19-550\19-550.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\19-551\19-551.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\19-552\19-552.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\19-553\19-553.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\19-554\19-554.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\19-555\19-555.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\19-556\19-556.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\19-557\19-557.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\19-558\19-558.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\19-559\19-559.

filePath is:D:\MWT\master_thesis\GCSAR\accepted\20-693\20-693.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\20-694\20-694.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\20-696\20-696.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\20-697\20-697.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\20-698\20-698.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\20-699\20-699.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\20-700\20-700.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\20-701\20-701.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\20-702\20-702.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\20-703\20-703.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\20-706\20-706.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\20-708\20-708.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\20-709\20-709.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\20-710\20-710.docx
filePath is:D:\MWT\master_thesis\GCSAR\accepted\20-711\20-711.

In [ ]:
# articles.describe()
articles.head()

,Article_ID,folder,ar_en,Country,Title,Author,Author_certificate,Author_specification,work,co_Author,...,Rev3_score,Score,Volume,Issue,Result,Date_of_editor_decision,artc_txt,clean_txt,clean2_txt,keywords
0,1,14-1,ar,Syria,تأثير طريقة الزراعة وعمق الحراثة في نمو الشوند...,فادي عباس,الدكتور,انتاج محاصيل (شوندر سكري),الهيئة العامة للبحوث العلمية الزراعية، مركز بح...,محمود السيدو,...,3.785714286,3.261904762,1,1,مقبول,NaN,تأثير طريقة الزراعة وعمق الحراثة في نمو الشوند...,NaN,NaN,NaN
1,3,14-3,ar,Syria,التأثير الطارد لمركب الكلس المكروني (Surround)...,مازن بوفاعور,المهندس,وقاية,الهيئة العامة للبحوث العلمية الزراعية، مركز بح...,"فيلي خاريزانوفا, ناريمان الزغبي",...,2,2.904761905,1,1,مقبول,NaN,التأثير الطارد لمركب الكلس المكروني (Surround)...,NaN,NaN,NaN
2,4,14-4,ar,Syria,تأثير طريقة الري بالتنقيط (مغطى ومكشوف) في الا...,عبد الكريم جردي,المهندس,ري محاصيل حقلية,الهيئة العامة للبحوث العلمية الزراعية، مركز بح...,"بسام عودة, هيثم العبدالله, طلال العبدو, بشرى خزام",...,3,3.452380952,2,2,مقبول,11/27/2014,تأثير طريقة الري بالتنقيط (مغطى ومكشوف) في الا...,NaN,NaN,NaN
3,5,14-5,ar,Syria,تغيرات مجتمع حشرة بسيلا الأجاص Cacopsylla pyri...,بسام عودة,الدكتور,وقاية نبات (حشرات),الهيئة العامة للبحوث العلمية الزراعية، مركز بح...,"وجيه قسيس, رندة أبو طارة",...,NaN,2.892857143,2,1,مقبول,NaN,تأثير المعقد الحيوي على مجتمعات حشرة بسيلا الأ...,NaN,NaN,NaN
4,6,14-6,ar,Syria,تأثير سبعة أصول من الحمضيات في مواصفات النمو و...,علاء سهيل ابراهيم,المهندس,علوم البستنة,الهيئة العامة للبحوث العلمية الزراعية، مركز بح...,"إشراق سليمان علي, رائد صبيح, رفيق علي عبود, فا...",...,2.642857143,2.880952381,1,1,مقبول,NaN,تأثير سبعة أصول من الحمضيات في مواصفات النمو و...,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,692,20-692,ar,Syria,المظاهر الوراثية المتعددة لإلفاكازين وعلاقتها ...,حسن المصري,الدكتور,إنتاج حيواني,"جامعة حلب, كلية الزراعة, قسم الإنتاج الحيواني",عبد الرحمن درويش، هبة البدعي,...,NaN,2.733333333,8,4,مقبول,22-3-2020,المظاهر الوراثية المتعددة لإلفاكازين وعلاقتها ...,NaN,NaN,NaN
496,693,20-693,ar,Syria,دور الزراعة الحافظة في زيادة إنتاجية محصول الق...,منال عثمان,الدكتورة,محاصيل حقلية,"جامعة دمشق, كلية الزراعة, قسم المحاصيل الحقلية","أيمن الشحاذه العوده, محمد منهل الزعبي",...,NaN,3.633333333,7,3,مقبول,NaN,تقييم دور الزراعة الحافظة في زيادة إنتاجية مح...,NaN,NaN,NaN
497,694,20-694,ar,Syria,تقدير القدرة على التوافق ودرجة السيادة وقوة ال...,محمد باقر العبد الواحد,الدكتور,محاصيل حقلية,"جامعة الفرات, كلية الزراعة, قسم المحاصيل الحقلية","أيمن العرفي, جلال شعبان عبود",...,NaN,2.133333333,7,3,مقبول,14-2-2020,تقدير القدرة على التوافق ودرجة السيادة وقوة ال...,NaN,NaN,NaN
498,696,20-696,ar,Iraq,استجابة أصناف الشوفان Avena sativa إلى التسميد...,لميا الفريح,الدكتورة,محاصيل حقلية,"جامعة البصرة, كلية الزراعة, قسم المحاصيل الحقلية","كاظم هذيلي, سندس العبدالله",...,NaN,2,7,2,مقبول,9/2/2020,استجابة اصناف الشوفان Avena sativa إلى التسميد...,NaN,NaN,NaN


In [ ]:
import re
import string
from camel_tools.tokenizers.word import simple_word_tokenize
from camel_tools.utils.dediac import dediac_ar
from nltk.corpus import stopwords
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
from farasa.stemmer import FarasaStemmer

In [ ]:
def remove_numbers_punctuations (text):
    clean_text = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', ' ', str(text))
    clean_text = re.sub(r'\w*\d\w*', ' ', str(clean_text))
    clean_text = re.sub(r'[%s]' % re.escape(string.punctuation), ' ', clean_text)
    clean_text = re.sub(r'،|؛', ' ', str(clean_text))
    return clean_text

In [ ]:
def tokenize(text):
    tokens = simple_word_tokenize(text)
    return tokens

In [ ]:
def remove_stop_words(tokens):
    filtered_text = [t for t in tokens if not t in stopwords.words("arabic")]
#     filtered_text = [t for t in filtered_text if not t in stopwords.words("english")]
    return filtered_text

In [ ]:
def remove_stop_words2(tokens):
    filtered_text = [t for t in tokens if not t in stopwords.words("english")]
    return filtered_text

In [ ]:
def ar_stemmer(text):
    stemmer = FarasaStemmer()
    stemmed_text = stemmer.stem(text)
    return stemmed_text


In [ ]:
def vectorizer(filtered_text):
    cv = CountVectorizer()
    transformed_text = cv.fit_transform(filtered_text)
    vector_table = pd.DataFrame(transformed_text.toarray(), columns=cv.get_feature_names())
    return vector_table

In [ ]:
def word_counter(word_list):
    counts = Counter(word_list)
    # Reverse the key/values in the dictionary for sorting
    word_counts = list(zip(counts.values(), counts.keys()))

    # Sort the list by count
    word_counts = sorted(word_counts, reverse=True)
    return word_counts

In [ ]:
def remove_single_double_char(text):
    no_singl_char =re.sub(r'[×©±≤≥]', ' ', str(text))
    no_singl_char =re.sub(r'\b\w{1,2}\b', ' ', str(no_singl_char))
#     simpols = '× , © , o , ≤ , ≥'
    return no_singl_char

In [ ]:
def only_ar_text(text):
    ar_text = re.sub(r"[A-Z]['\w]*|[a-z]['\w]*", ' ', str(text))
    return ar_text

In [ ]:
articles['clean_txt'] = articles.artc_txt.map(remove_numbers_punctuations)
# .map(tokenize).map(remove_stop_words)

In [ ]:
articles['clean_txt'] = articles['clean_txt'].map(tokenize)

In [ ]:
articles['clean_txt'] = articles['clean_txt'].map(remove_stop_words)

KeyboardInterrupt: 

In [ ]:
articles.clean_txt[0:100]

0     [تأثير, طريقة, الزراعة, وعمق, الحراثة, في, نمو...
1     [التأثير, الطارد, لمركب, الكلس, المكروني, Surr...
2     [تأثير, طريقة, الري, بالتنقيط, مغطى, ومكشوف, ف...
3     [تأثير, المعقد, الحيوي, على, مجتمعات, حشرة, بس...
4     [تأثير, سبعة, أصول, من, الحمضيات, في, مواصفات,...
                            ...                        
95    [تأثير, التسميد, العضوي, ومعدلات, التسميد, الم...
96    [تأثير, قطع, الشماريخ, الزهرية, في, الشوندر, ا...
97    [دراسة, أولية, لنبات, القلقاس, البري, الملخص, ...
98    [تأثير, نموذج, ولادة, الأمات, والآباء, عمر, ال...
99    [اقتصاديات, إنتاج, الزيتون, البعل, في, محافظة,...
Name: clean_txt, Length: 100, dtype: object

In [ ]:
for i,txt in enumerate(articles.clean2_txt):
    articles.clean2_txt[i] = remove_stop_words2(articles.clean2_txt[i])
    print(i,'done')


0 done
1 done
2 done
3 done
4 done
5 done
6 done
7 done
8 done
9 done
10 done
11 done
12 done
13 done
14 done
15 done
16 done
17 done
18 done
19 done
20 done
21 done
22 done
23 done
24 done
25 done
26 done
27 done
28 done
29 done
30 done
31 done
32 done
33 done
34 done
35 done
36 done
37 done
38 done
39 done
40 done
41 done
42 done
43 done
44 done
45 done
46 done
47 done
48 done
49 done
50 done
51 done
52 done
53 done
54 done
55 done
56 done
57 done
58 done
59 done
60 done
61 done
62 done
63 done
64 done
65 done
66 done
67 done
68 done
69 done
70 done
71 done
72 done
73 done
74 done
75 done
76 done
77 done
78 done
79 done
80 done
81 done
82 done
83 done
84 done
85 done
86 done
87 done
88 done
89 done
90 done
91 done
92 done
93 done
94 done
95 done
96 done
97 done
98 done
99 done
100 done
101 done
102 done
103 done
104 done
105 done
106 done
107 done
108 done
109 done
110 done
111 done
112 done
113 done
114 done
115 done
116 done
117 done
118 done
119 done
120 done
121 done
122 done
123

In [ ]:
articles['clean_txt'] = articles['clean_txt'].map(remove_stop_words2)

KeyboardInterrupt: 

In [ ]:
articles.tail()

,Article_ID,folder,ar_en,Country,Title,Author,Author_certificate,Author_specification,work,co_Author,...,Volume,Issue,Result,Date_of_editor_decision,artc_txt,clean_txt,clean2_txt,keywords,clean3_txt,ar_keywords
584,818,20-818,ar,Syria,عزل وتصنيف الليستريا Listeria.spp في لحوم الأغ...,فراس سلطان,الدكتور,صحة الحيوان,"جامعة حماة, كلية الطب البيطري, قسم الصحة العام...","دارم طباع, عبد العزيز عروانه",...,8,1,مقبول,18-7-2020,عزل وتصنيف الليستريا في لحوم الأغنام في مدينة ...,"[عزل, وتصنيف, الليستريا, في, لحوم, الأغنام, في...","[عزل, وتصنيف, الليستريا, لحوم, الأغنام, مدينة,...","[(58, ليستريا), (44, لحم), (44, Listeria), (34...",عزل تصنيف ليستريا لحم غنم مدينة ملخص أجرى در...,"[(58, 'ليستريا'), (44, 'لحم'), (44, ' ), (34, ..."
585,819,20-819,ar,Syria,أثر سياسة التحرر الاقتصادي في المؤشرات الإنتاج...,اسماعيل عمران,الدكتور,اقتصاد زراعي,"جامعة تشرين, كلية الزراعة, قسم الاقتصاد الزراعي","غسان يعقوب, محمود عليو",...,8,4,مقبول,NaN,أثر سياسة التحرر الاقتصادي على المؤشرات الإنتا...,"[أثر, سياسة, التحرر, الاقتصادي, على, المؤشرات,...","[أثر, سياسة, التحرر, الاقتصادي, المؤشرات, الإن...","[(105, اقتصادي), (90, تحرر), (88, إنتاج), (84,...",أثر سياسة تحرر اقتصادي مؤشر إنتاجية اقتصادي مح...,"[(105, 'اقتصادي'), (90, 'تحرر'), (88, 'إنتاج')..."
586,821,20-821,ar,Syria,تقدير كفاءة إنتاج لحم الفروج باستخدام تحليل مغ...,مايا العبدالله,الدكتورة,اقتصاد زراعي,الهيئة العامة للبحوث العلمية الزراعية، مركز بح...,صفوان أبوعساف، عفراء سلوم,...,8,3,مقبول,2/7/2020,تقدير كفاءة إنتاج لحم الفروج باستخدام تحليل مغ...,"[تقدير, كفاءة, إنتاج, لحم, الفروج, باستخدام, ت...","[تقدير, كفاءة, إنتاج, لحم, الفروج, باستخدام, ت...","[(120, كفاءة), (68, مداجن), (61, إنتاج), (54, ...",تقدير كفاءة إنتاج لحم فرج استخدام تحليل مغلف ب...,"[(120, 'كفاءة'), (68, 'مداجن'), (61, 'إنتاج'),..."
587,822,20-822,ar,Libya,القدرة التضادية لعزلة محلية من الفطر Trichode...,آمنة سعد,المهندسة,وقاية النبات,"جامعة عمر المختار, كلية الزراعة, قسم وقاية النبات",زهرة الجالي,...,7,4,مقبول,NaN,دراسة مُختبرية لتثبيط بعض الفطريات المُمْرضة ل...,"[دراسة, مُختبرية, لتثبيط, بعض, الفطريات, المُم...","[دراسة, مُختبرية, لتثبيط, الفطريات, المُمْرضة,...","[(144, فطر), (80, ممرض), (62, فطري), (61, نمو)...",دراسة مختبري تثبيط فطري ممرض نبات واسطة آلي مخ...,"[(144, 'فطر'), (80, 'ممرض'), (62, 'فطري'), (61..."
588,823,20-823,ar,Iraq,دراسة تقييم كفاءة المعالجات الحيوية والنانوية ...,صائب هاشم رسن,الدكتور,علوم البستنة,"جامعة البصرة, كلية الزراعة",ابتهاج التميمي,...,7,6,مقبول,1/9/2020,دراسة تقييم كفاءة المعالجات الحيوية والنانوية ...,"[دراسة, تقييم, كفاءة, المعالجات, الحيوية, والن...","[دراسة, تقييم, كفاءة, المعالجات, الحيوية, والن...","[(113, إضافة), (107, معاملة), (85, معالج), (85...",دراسة تقييم كفاءة معالجة حيوية نانوي طريق أضاف...,"[(113, 'إضافة'), (107, 'معاملة'), (85, 'معالج'..."


In [ ]:
articles.to_csv(r'C:\Users\User\Dropbox\PC\Documents\python\14.csv' ,sep="#")

In [ ]:
joiner = lambda x : " ".join(x)

In [ ]:
articles['clean3_txt'] = articles['keywords']

In [ ]:
articles['clean3_txt'] = articles['clean2_txt'].map(joiner)

In [ ]:
articles['clean3_txt'] = articles['clean3_txt'].map(dediac_ar)

In [ ]:
articles['clean3_txt'] = articles['clean2_txt'].map(ar_stemmer)

KeyboardInterrupt: 

In [ ]:
for i,txt in enumerate(articles.clean3_txt):
    articles.clean3_txt[i] = ar_stemmer(articles.clean3_txt[i])
    print(i,'done')


C:\Users\User\AppData\Local\Temp/ipykernel_11064/3400400509.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  articles.clean3_txt[i] = ar_stemmer(articles.clean3_txt[i])


0 done
1 done
2 done
3 done
4 done
5 done
6 done
7 done
8 done
9 done
10 done
11 done
12 done
13 done
14 done
15 done
16 done
17 done
18 done
19 done
20 done
21 done
22 done
23 done
24 done
25 done
26 done
27 done
28 done
29 done
30 done
31 done
32 done
33 done
34 done
35 done
36 done
37 done
38 done
39 done
40 done
41 done
42 done
43 done
44 done
45 done
46 done
47 done
48 done
49 done
50 done
51 done
52 done
53 done
54 done
55 done
56 done
57 done
58 done
59 done
60 done
61 done
62 done
63 done
64 done
65 done
66 done
67 done
68 done
69 done
70 done
71 done
72 done
73 done
74 done
75 done
76 done
77 done
78 done
79 done
80 done
81 done
82 done
83 done
84 done
85 done
86 done
87 done
88 done
89 done
90 done
91 done
92 done
93 done
94 done
95 done
96 done
97 done
98 done
99 done
100 done
101 done
102 done
103 done
104 done
105 done
106 done
107 done
108 done
109 done
110 done
111 done
112 done
113 done
114 done
115 done
116 done
117 done
118 done
119 done
120 done
121 done
122 done
123

In [ ]:
articles['clean3_txt'] = articles['clean3_txt'].map(remove_single_double_char)

In [ ]:
articles['keywords'] = articles['clean3_txt'].map(tokenize).map(word_counter)

In [ ]:
articles['ar_keywords'] = articles['folder']

In [ ]:
for i,txt in enumerate(articles.ar_keywords):
    articles.ar_keywords[i] = only_ar_text(articles.keywords[i])
    print(i,'done')


C:\Users\User\AppData\Local\Temp/ipykernel_11064/3588258571.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  articles.ar_keywords[i] = only_ar_text(articles.keywords[i])


0 done
1 done
2 done
3 done
4 done
5 done
6 done
7 done
8 done
9 done
10 done
11 done
12 done
13 done
14 done
15 done
16 done
17 done
18 done
19 done
20 done
21 done
22 done
23 done
24 done
25 done
26 done
27 done
28 done
29 done
30 done
31 done
32 done
33 done
34 done
35 done
36 done
37 done
38 done
39 done
40 done
41 done
42 done
43 done
44 done
45 done
46 done
47 done
48 done
49 done
50 done
51 done
52 done
53 done
54 done
55 done
56 done
57 done
58 done
59 done
60 done
61 done
62 done
63 done
64 done
65 done
66 done
67 done
68 done
69 done
70 done
71 done
72 done
73 done
74 done
75 done
76 done
77 done
78 done
79 done
80 done
81 done
82 done
83 done
84 done
85 done
86 done
87 done
88 done
89 done
90 done
91 done
92 done
93 done
94 done
95 done
96 done
97 done
98 done
99 done
100 done
101 done
102 done
103 done
104 done
105 done
106 done
107 done
108 done
109 done
110 done
111 done
112 done
113 done
114 done
115 done
116 done
117 done
118 done
119 done
120 done
121 done
122 done
123

In [ ]:
# saving models and dataframes
import joblib
# or from sklearn.externals import joblib

In [ ]:
joblib.dump(articles, "article_scope_model.pkl")

['article_scope_model.pkl']